# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)
print(mongo.list_database_names())

['Mongo_class', 'admin', 'classDB', 'config', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

In [6]:
pprint(db.establishments.find_one())

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('642b60ffa68a98f09c8c67be'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
# Find the establishments with a hygiene score of 20
filter = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
hygiene_score = establishments.count_documents(filter)

print(f"There are {hygiene_score} establishments with a hygiene score of 20.")

There are 0 establishments with a hygiene score of 20.


In [8]:
# Display the first document in the results using pprint
pprint(establishments.find_one(hygiene_score))

None


In [9]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(establishments.find(hygiene_score))

# Display the number of rows in the DataFrame
print(f"There are {len(hygiene_df)} rows in this DataFrame.")

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

There are 3 rows in this DataFrame.


,_id,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,Phone,LocalAuthorityCode,LocalAuthorityName,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending
0,642b60ffa68a98f09c8c67be,Penang Flavours,Restaurant/Cafe/Canteen,1,Penang Flavours,146A Plumstead Rd,London,,SE18 7DY,,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': '', 'Structural': '', 'ConfidenceI...",FHRS,"{'longitude': 0.08384, 'latitude': 51.490142}",,4623.972328,True
1,6444b18a728014f48da0e401,Penang Flavours,Restaurant/Cafe/Canteen,,Penang Flavours,146A Plumstead Rd,London,,SE18 7DY,,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': '', 'Structural': '', 'ConfidenceI...",FHRS,"{'longitude': 0.08384, 'latitude': 51.490142}",,4623.972328,True
2,6444c25d728014f48da0e403,Penang Flavours,Restaurant/Cafe/Canteen,1,Penang Flavours,146A Plumstead Rd,London,,SE18 7DY,,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': '', 'Structural': '', 'ConfidenceI...",FHRS,"{'longitude': 0.08384, 'latitude': 51.490142}",,4623.972328,True


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [10]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
London_query = {'LocalAuthorityName': {'$regex': 'London'}, 
          'RatingValue': {'$gte': '4'}
         }

# Use count_documents to display the number of documents in the result
London_count = establishments.count_documents(London_query)
print(f"There are {London_count} establishments in London that have a RatingValue greater than or equal to 4.")

# Display the first document in the results using pprint


There are 0 establishments in London that have a RatingValue greater than or equal to 4.


In [11]:
# Convert the result to a Pandas DataFrame
london_df = pd.DataFrame(establishments.find(London_query))

# Display the number of rows in the DataFrame
print(f"There are {len(london_df)} rows in this DataFrame.")

# Display the first 10 rows of the DataFrame
london_df.head(10)

There are 0 rows in this DataFrame.


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [12]:
pprint(establishments.find_one({'BusinessName': 'Penang Flavours'}, {'geocode.latitude', 'geocode.longitude'}))

{'_id': ObjectId('642b60ffa68a98f09c8c67be'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384}}


In [13]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142  
longitude = 0.08384

query = {'RatingValue': '5', 
         'geocode.latitude': {'$lte': (latitude + degree_search), '$gte': (latitude - degree_search)}, 
         'geocode.longitude': {'$lte': (longitude + degree_search), '$gte': (longitude - degree_search)}
        }

sort =  [('scores.Hygiene', 1)]

# Print the results
for result in establishments.find(query).sort(sort).limit(5):
    pprint(result)

In [14]:
# Convert result to Pandas DataFrame
pd.DataFrame(establishments.find(query).sort(sort).limit(5))

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [15]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match': {'scores.Hygiene': 0}}
group_query = {'$group': {'_id': '$LocalAuthorityName', 
                          'count': {'$sum': 1}
                         }
              }
sort_query = {'$sort': {'count': -1}}

pipeline = [match_query, group_query, sort_query]

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"There are {len(results)} documents in the result.")

# Print the first 10 results
pprint(results[0:10])

There are 0 documents in the result.
[]


In [16]:
# Convert the result to a Pandas DataFrame
agg_df = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(f"There are {len(agg_df)} rows in the DataFrame.")

# Display the first 10 rows of the DataFrame
agg_df.head(10)

There are 0 rows in the DataFrame.


""
